In [1]:
import tensorflow as tf

print('TF version:', tf.__version__)

# Check if GPU is available
if tf.test.gpu_device_name() is not None:
    print('GPU available')
else:
    print('Not using GPU')


2023-05-28 05:37:50.467056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF version: 2.12.0
GPU available


In [2]:
import numpy as np
import pandas as pd
import os
from re import search
import shutil
import natsort
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import imageio

seed = 7
np.random.seed(seed)

In [3]:
folder = '2020_JOCN_constellation_dataset' # folder where the dataset is located

points_per_symbol = 300 # points per symbol to use during the training

# scenarios available in the dataset
scenarios = ['4QAM_OSNR12dB', '4QAM_OSNR14dB', '4QAM_OSNR16dB', '4QAM_OSNR18dB', 
             '4QAM_OSNR20dB', '4QAM_OSNR22dB',
             '4QAM_OSNR24dB', '4QAM_OSNR26dB', '4QAM_OSNR28dB', '4QAM_OSNR30dB',
             '16QAM_OSNR20dB', '16QAM_OSNR22dB',
             '16QAM_OSNR24dB', '16QAM_OSNR26dB', '16QAM_OSNR28dB', '16QAM_OSNR30dB', 
             '16QAM_OSNR32dB', '16QAM_OSNR34dB',
             '16QAM_OSNR36dB', '16QAM_OSNR38dB',
             '64QAM_OSNR26dB', '64QAM_OSNR28dB', '64QAM_OSNR30dB', '64QAM_OSNR32dB',
             '64QAM_OSNR34dB', '64QAM_OSNR36dB', '64QAM_OSNR38dB', '64QAM_OSNR40dB', 
             '64QAM_OSNR42dB', '64QAM_OSNR44dB']

In [4]:
evm_error_metrics = {'mean_absolute_error': {}, 'mean_squared_error': {}, 
                     'mean_squared_logarithmic_error': {}}
evm_truth = {}


ber_error_metrics = {'mean_absolute_error': {}, 'mean_squared_error': {}, 
                     'mean_squared_logarithmic_error': {}}
ber_truth = {}

In [5]:
import scipy.io as sio
import tqdm as tqdm
number_figures = {}
for scenario in tqdm.tqdm(scenarios, desc='Reading the ground truth'):
    evm_error_metrics['mean_absolute_error'][scenario] = {}
    evm_error_metrics['mean_squared_error'][scenario] = {}
    evm_error_metrics['mean_squared_logarithmic_error'][scenario] = {}
    
    ber_error_metrics['mean_absolute_error'][scenario] = {}
    ber_error_metrics['mean_squared_error'][scenario] = {}
    ber_error_metrics['mean_squared_logarithmic_error'][scenario] = {}
    
    # loading the truth value
    mat_contents = sio.loadmat(os.path.join(folder, 'fullTrace', scenario, 
                                'OutputParams_' + scenario + '_FullTrace.mat'))
    evm_truth[scenario] = mat_contents['output']['EVM_mu'][0][0]
    ber_truth[scenario] = mat_contents['output']['BERe_mu'][0][0]
    del mat_contents
    
    for file_mat in sorted(os.listdir(os.path.join(folder, scenario))):
        if file_mat.endswith('.mat') and f'PointsPerSymb_{points_per_symbol}' in file_mat:
            names = file_mat.split('_')
            modulation = names[1]
            osnr = names[2]
            points_per_symbol = int(names[4].replace('.mat', ''))
            
            # import matlab file
            mat_contents = sio.loadmat(os.path.join(folder, scenario, file_mat))
            number_figures[scenario] = 100 #len(evm_figures)
            
            evm_truth_vector = np.tile(evm_truth[scenario], number_figures[scenario])
            ber_truth_vector = np.tile(ber_truth[scenario], number_figures[scenario])
            del mat_contents

Reading the ground truth: 100%|██████████| 30/30 [00:09<00:00,  3.17it/s]


In [6]:
dx = 40
dx1 = 60
dy = 50

cut_x = 68+dx
cut_x1 = 68+dx1
cut_y = 168+dy

X_train = np.zeros((len(scenarios) * 50,  520-dx-dx1, 520-2*dy))
Y_train = np.zeros((len(scenarios) * 50))

X_validation = np.zeros((len(scenarios) * 25,  520-dx-dx1, 520-2*dy))
Y_validation = np.zeros((len(scenarios) * 25))

X_test = np.zeros((len(scenarios) * 25,  520-dx-dx1, 520-2*dy))
Y_test = np.zeros((len(scenarios) * 25))

pbar = tqdm.tqdm(total=len(scenarios)*100, desc='Loading constellation diagrams')

for id_scenario, scenario in enumerate(scenarios):

    index_full = id_scenario * 100
    
    index_train = id_scenario * 50
    index_validation = id_scenario * 25
    index_test = id_scenario * 25

    for id_figure in range(number_figures[scenario]):
        image = imageio.imread(os.path.join(folder, scenario,
                                            'Fig_'+ scenario + '_PointsPerSymb_' + str(points_per_symbol) + '_constN_' + str(id_figure + 1) + '.png.png'))
        shape = image.shape
        
        r, g, b = image[cut_x:shape[0]-cut_x1, cut_y+20:shape[1] - cut_y + 1, 0], image[cut_x:shape[0]-cut_x1, cut_y+20:shape[1] - cut_y + 1, 1], image[cut_x:shape[0]-cut_x1, cut_y+20:shape[1] - cut_y + 1, 2]
     
        gray = (0.2989 * r + 0.5870 * g + 0.1140 * b) / 255.
        index = id_scenario * 100 + id_figure

        # importing it either to training, validation or test datasets
        if id_figure < 50:
            X_train[index_train + id_figure, :, :] = gray
            Y_train[index_train + id_figure] = evm_truth[scenario]
            # print('train', index_train + id_figure, '/', len(scenarios)*50)
        elif id_figure < 75:
            X_validation[index_validation + id_figure - 50, :, :] = gray
            Y_validation[index_validation + id_figure - 50] = evm_truth[scenario]
            # print('validation', index_validation + id_figure - 50, '/', len(scenarios)*25)
        else:
            X_test[index_test + id_figure - 75, :, :] = gray
            Y_test[index_test + id_figure - 75] = evm_truth[scenario]
            # print('test', index_test + id_figure - 75, '/', len(scenarios)*25)
        pbar.update()

Loading constellation diagrams:   0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_947125/3943943973.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(os.path.join(folder, scenario,
Loading constellation diagrams: 100%|██████████| 3000/3000 [01:04<00:00, 79.43it/s]

In [8]:
import os
import warnings

import imageio
import numpy as np

import scipy.io as sio

from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error

import matplotlib.pyplot as plt

Loading constellation diagrams: 100%|██████████| 3000/3000 [00:51<00:00, 92.56it/s]

In [8]:
X_trainset1= X_train[:750,:,:]
X_trainset2= X_train[750:1075,:,:]

Y_trainset1=Y_train[:750]
Y_trainset2= Y_train[750:1075]


In [22]:
X_testset1 = X_test[:325, :, :]
X_testset2 = X_test[325:487, :, :]

Y_testset1 = Y_test[:325]
Y_testset2 = Y_test[325:487]


In [23]:

X_validationset1 = X_test[:325, :, :]
X_validationset2 = X_test[325:487, :, :]

Y_validationset1 = Y_test[:325]
Y_validationset2 = Y_test[325:487]


In [12]:
X_train.shape


(1500, 420, 420)

In [13]:
Y_train.shape



(1500,)

In [14]:
X_trainset1.shape


(750, 420, 420)

In [15]:
Y_trainset1.shape


(750,)

In [16]:
X_trainset2.shape


(325, 420, 420)

In [17]:
Y_trainset2.shape

(325,)

In [24]:
X_test.shape



(750, 420, 420)

In [25]:
Y_test.shape


(750,)

In [26]:
X_testset1.shape


(325, 420, 420)

In [27]:
Y_testset1.shape


(325,)

In [28]:
X_testset2.shape


(162, 420, 420)

In [29]:
Y_testset2.shape

(162,)

In [30]:
X_validation.shape


(750, 420, 420)

In [31]:
Y_validation.shape


(750,)

In [32]:
X_validationset1.shape


(325, 420, 420)

In [33]:
Y_validationset1.shape



(325,)

In [34]:
X_validationset2.shape


(162, 420, 420)

In [35]:
Y_validationset2.shape

(162,)

In [53]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.python.keras import optimizers as opt
from tensorflow.keras.optimizers import Adam

In [54]:
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
# Convolutional Classifier Regressor
model_folder = 'conv_classifier_regressor'
results_folder = os.path.join(folder, 'results', model_folder)
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)
    print('created folder', results_folder)

input_0 = Input(shape=(X_train.shape[1], X_train.shape[2], 1))
conv_0 = Conv2D(4, kernel_size=(3, 3), activation='relu')(input_0)
pool_0 = MaxPooling2D(pool_size=(2, 2))(conv_0)

conv_1 = Conv2D(8, kernel_size=(3, 3), activation='relu')(pool_0)
pool_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)

conv_2 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)

conv_3 = Conv2D(32, kernel_size=(3, 3), activation='relu')(pool_2)
pool_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)

flat_0 = Flatten()(pool_3)
dense_0 = Dense(1024, activation='relu')(flat_0)
dense_1_0 = Dense(512, activation='relu',name='bisect')(dense_0)

out_0 = Dense(1, name='evm')(dense_1_0)
cnn_model = Model(inputs=input_0, outputs=out_0)
cnn_model.compile(optimizer=optimizers.Adam(lr=1e-04), loss='msle')
cnn_model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 420, 420, 1)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 418, 418, 4)       40        
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 209, 209, 4)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 207, 207, 8)       296       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 103, 103, 8)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 101, 101, 16)      1168

In [55]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

# Splitting the data into train and test sets
X_train_cnn, X_test_cnn, Y_train_cnn, Y_test_cnn = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Train the CNN model
cnn_model.fit(X_train_cnn, Y_train_cnn, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate CNN model
Y_pred_cnn = cnn_model.predict(X_test_cnn)

mse_cnn = mean_squared_error(Y_test_cnn, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test_cnn, Y_pred_cnn)
r2_cnn = r2_score(Y_test_cnn, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test_cnn, Y_pred_cnn)


print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)






Epoch 1/30
30/30 [==============================] - 12s 371ms/step - loss: 0.4780 - val_loss: 0.1427
Epoch 2/30
30/30 [==============================] - 11s 354ms/step - loss: 0.1035 - val_loss: 0.0382
Epoch 3/30
30/30 [==============================] - 10s 344ms/step - loss: 0.0188 - val_loss: 0.0050
Epoch 4/30
30/30 [==============================] - 10s 343ms/step - loss: 0.0052 - val_loss: 0.0033
Epoch 5/30
30/30 [==============================] - 11s 363ms/step - loss: 0.0046 - val_loss: 0.0031
Epoch 6/30
30/30 [==============================] - 11s 370ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 7/30
30/30 [==============================] - 11s 353ms/step - loss: 0.0021 - val_loss: 0.0013
Epoch 8/30
30/30 [==============================] - 10s 346ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 9/30
30/30 [==============================] - 10s 341ms/step - loss: 3.6341e-04 - val_loss: 2.9330e-04
Epoch 25/30
30/30 [==============================] - 10s 346ms/step - loss: 2.9967e

In [56]:
from tensorflow.keras.models import Model
model_feat = Model(inputs=cnn_model.input,outputs=cnn_model.get_layer("bisect").output)

In [57]:
feat_train = model_feat.predict(X_train)
print(feat_train.shape)

47/47 [==============================] - 3s 53ms/step
(1500, 512)


In [58]:
feat_test = model_feat.predict(X_test)
print(feat_test.shape)

24/24 [==============================] - 1s 46ms/step
(750, 512)


In [59]:
from timeit import default_timer as timer

In [60]:
from sklearn.svm import SVR
# Create SVR model
svr_model = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svr_model.fit(feat_train, Y_train)

# start time
start = timer()
svr_predictions_train = svr_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ",end - start)

Time taken to predict with trained data:  0.12424509041011333


In [61]:
import xgboost as xgb

# Create XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(feat_train, Y_train)

# start time
start = timer()
xgb_predictions_train = xgb_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.002720113843679428


In [62]:
from sklearn.ensemble import RandomForestRegressor

# Create Random Forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(feat_train, Y_train)

# start time
start = timer()
rf_predictions_train = rf_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.01513587310910225


In [63]:
from sklearn.tree import DecisionTreeRegressor

# Create Decision Tree model
dt_model = DecisionTreeRegressor()
dt_model.fit(feat_train, Y_train)

# start time
start = timer()
dt_predictions_train = dt_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.00048143230378627777


In [64]:
from sklearn.linear_model import SGDRegressor

# Create SGD model
sgd_model = SGDRegressor(loss='epsilon_insensitive', epsilon=0.1, max_iter=1000, tol=1e-3)
sgd_model.fit(feat_train, Y_train)

# start time
start = timer()
sgd_predictions_train = sgd_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.0013573896139860153


In [65]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


# SVR model

svr_predictions = svr_model.predict(feat_test)


# Concatenate SVR predictions with CNN predictions
combined_predictions = np.column_stack((Y_test, svr_predictions))

mse_combined = mean_squared_error(Y_test, svr_predictions)
mae_combined = mean_absolute_error(Y_test, svr_predictions)
r2_combined = r2_score(Y_test, svr_predictions)
evs_combined = explained_variance_score(Y_test, svr_predictions)


print("CNN + SVR Model Performance:")
print("Mean Squared Error:", mse_combined)
print("Mean Absolute Error:", mae_combined)
print("R-squared:", r2_combined)
print("Explained Variance Score:", evs_combined)

24/24 [==============================] - 1s 50ms/step
CNN Model Performance:
Mean Squared Error: 0.07507867487671616
Mean Absolute Error: 0.1335667706726129
R-squared: 0.9981025880689965
Explained Variance Score: 0.998173330066856
CNN + SVR Model Performance:
Mean Squared Error: 0.029607964804956696
Mean Absolute Error: 0.10096115218635612
R-squared: 0.9992517381831005
Explained Variance Score: 0.9992517405433268


In [66]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


import xgboost as xgb

# XGBoost model
xgb_predictions = xgb_model.predict(feat_test)

mse_xgb = mean_squared_error(Y_test, xgb_predictions)
mae_xgb = mean_absolute_error(Y_test, xgb_predictions)
r2_xgb = r2_score(Y_test, xgb_predictions)
evs_xgb = explained_variance_score(Y_test, xgb_predictions)

print("CNN + XGBoost Model Performance:")
print("Mean Squared Error:", mse_xgb)
print("Mean Absolute Error:", mae_xgb)
print("R-squared:", r2_xgb)
print("Explained Variance Score:", evs_xgb)


24/24 [==============================] - 1s 46ms/step
CNN Model Performance:
Mean Squared Error: 0.07507867487671616
Mean Absolute Error: 0.1335667706726129
R-squared: 0.9981025880689965
Explained Variance Score: 0.998173330066856
CNN + XGBoost Model Performance:
Mean Squared Error: 0.007144036871676732
Mean Absolute Error: 0.018741546860064053
R-squared: 0.9998194536488809
Explained Variance Score: 0.9998207816511249


In [67]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.ensemble import RandomForestRegressor

# Random Forest model
rf_predictions = rf_model.predict(feat_test)

mse_rf = mean_squared_error(Y_test, rf_predictions)
mae_rf = mean_absolute_error(Y_test, rf_predictions)
r2_rf = r2_score(Y_test, rf_predictions)
evs_rf = explained_variance_score(Y_test, rf_predictions)

print("CNN + Random Forest Model Performance:")
print("Mean Squared Error:", mse_rf)
print("Mean Absolute Error:", mae_rf)
print("R-squared:", r2_rf)
print("Explained Variance Score:", evs_rf)


24/24 [==============================] - 1s 54ms/step
CNN Model Performance:
Mean Squared Error: 0.07507867487671616
Mean Absolute Error: 0.1335667706726129
R-squared: 0.9981025880689965
Explained Variance Score: 0.998173330066856
CNN + Random Forest Model Performance:
Mean Squared Error: 0.008413143252765293
Mean Absolute Error: 0.01618478043523932
R-squared: 0.9997873803924849
Explained Variance Score: 0.9997886277913237


In [68]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.tree import DecisionTreeRegressor

# Decision Tree model
dt_predictions = dt_model.predict(feat_test)

mse_dt = mean_squared_error(Y_test, dt_predictions)
mae_dt = mean_absolute_error(Y_test, dt_predictions)
r2_dt = r2_score(Y_test, dt_predictions)
evs_dt = explained_variance_score(Y_test, dt_predictions)

print("CNN + Decision Tree Model Performance:")
print("Mean Squared Error:", mse_dt)
print("Mean Absolute Error:", mae_dt)
print("R-squared:", r2_dt)
print("Explained Variance Score:", evs_dt)


24/24 [==============================] - 1s 52ms/step
CNN Model Performance:
Mean Squared Error: 0.07507867487671616
Mean Absolute Error: 0.1335667706726129
R-squared: 0.9981025880689965
Explained Variance Score: 0.998173330066856
CNN + Decision Tree Model Performance:
Mean Squared Error: 0.014591938632264543
Mean Absolute Error: 0.009919088661027408
R-squared: 0.9996312279285324
Explained Variance Score: 0.9996315657610321


In [69]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

from sklearn.linear_model import SGDRegressor

# SGD model
sgd_predictions = sgd_model.predict(feat_test)

mse_sgd = mean_squared_error(Y_test, sgd_predictions)
mae_sgd = mean_absolute_error(Y_test, sgd_predictions)
r2_sgd = r2_score(Y_test, sgd_predictions)
evs_sgd = explained_variance_score(Y_test, sgd_predictions)

print("CNN + SGD Model Performance:")
print("Mean Squared Error:", mse_sgd)
print("Mean Absolute Error:", mae_sgd)
print("R-squared:", r2_sgd)
print("Explained Variance Score:", evs_sgd)


24/24 [==============================] - 1s 47ms/step
CNN Model Performance:
Mean Squared Error: 0.07507867487671616
Mean Absolute Error: 0.1335667706726129
R-squared: 0.9981025880689965
Explained Variance Score: 0.998173330066856
CNN + SGD Model Performance:
Mean Squared Error: 0.0700190753152954
Mean Absolute Error: 0.12885361413227583
R-squared: 0.9982304558635428
Explained Variance Score: 0.9982598559878992


In [70]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import optimizers

# Define the model and results folder
model_folder = 'conv_classifier_regressor'
results_folder = os.path.join(folder, 'results', model_folder)
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)
    print('Created folder', results_folder)

# Define the input shape and model architecture
input_0 = Input(shape=(X_trainset1.shape[1], X_trainset1.shape[2], 1))
conv_0 = Conv2D(4, kernel_size=(3, 3), activation='relu')(input_0)
pool_0 = MaxPooling2D(pool_size=(2, 2))(conv_0)

conv_1 = Conv2D(8, kernel_size=(3, 3), activation='relu')(pool_0)
pool_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)

conv_2 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)

conv_3 = Conv2D(32, kernel_size=(3, 3), activation='relu')(pool_2)
pool_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)

flat_0 = Flatten()(pool_3)
dense_0 = Dense(1024, activation='relu')(flat_0)
dense_1_0 = Dense(512, activation='relu', name='bisect')(dense_0)

out_0 = Dense(1, name='evm')(dense_1_0)
cnn_model = Model(inputs=input_0, outputs=out_0)
cnn_model.compile(optimizer=optimizers.Adam(lr=1e-04), loss='msle')
cnn_model.summary()

# Split the data into train and test sets
X_trainset1, X_testset1, Y_trainset1, Y_testset1 = train_test_split(X_trainset1, Y_trainset1, test_size=0.2, random_state=42)

# Train the CNN model
cnn_model.fit(X_trainset1, Y_trainset1, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the CNN model
Y_pred_cnn = cnn_model.predict(X_testset1)

mse_cnn = mean_squared_error(Y_testset1, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_testset1, Y_pred_cnn)
r2_cnn = r2_score(Y_testset1, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_testset1, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 420, 420, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 418, 418, 4)       40        
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 209, 209, 4)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 207, 207, 8)       296       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 103, 103, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 101, 101, 16)      1168

In [71]:
model_feat = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer("bisect").output)
feat_train = model_feat.predict(X_trainset1)
print(feat_train.shape)

15/15 [==============================] - 1s 45ms/step
(480, 512)


In [72]:
feat_test = model_feat.predict(X_testset1)
print(feat_test.shape)

4/4 [==============================] - 0s 51ms/step
(120, 512)


In [73]:
from tensorflow.keras.models import Model
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from timeit import default_timer as timer
# Create SVR model
svr_model = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svr_model.fit(feat_train, Y_trainset1)

# start time
start = timer()
svr_predictions_train = svr_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.022470945492386818


In [74]:
# Create XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(feat_train, Y_trainset1)

# start time
start = timer()
xgb_predictions_train = xgb_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.0017111040651798248


In [75]:
# Create Random Forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(feat_train, Y_trainset1)

# start time
start = timer()
rf_predictions_train = rf_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.009448304772377014


In [76]:
# Create Decision Tree model
dt_model = DecisionTreeRegressor()
dt_model.fit(feat_train, Y_trainset1)

# start time
start = timer()
dt_predictions_train = dt_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.0003116857260465622


In [77]:
# Create SGD model
sgd_model = SGDRegressor(loss='epsilon_insensitive', epsilon=0.1, max_iter=1000, tol=1e-3)
sgd_model.fit(feat_train, Y_trainset1)

# start time
start = timer()
sgd_predictions_train = sgd_model.predict(feat_train)
# end time
end = timer()
print("Time taken to predict with trained data: ", end - start)

Time taken to predict with trained data:  0.0009936951100826263


In [78]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor

# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_testset1)

mse_cnn = mean_squared_error(Y_testset1, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_testset1, Y_pred_cnn)
r2_cnn = r2_score(Y_testset1, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_testset1, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


# SVR model
svr_predictions = svr_model.predict(feat_test)

mse_svr = mean_squared_error(Y_testset1, svr_predictions)
mae_svr = mean_absolute_error(Y_testset1, svr_predictions)
r2_svr = r2_score(Y_testset1, svr_predictions)
evs_svr = explained_variance_score(Y_testset1, svr_predictions)

print("CNN + SVR Model Performance:")
print("Mean Squared Error:", mse_svr)
print("Mean Absolute Error:", mae_svr)
print("R-squared:", r2_svr)
print("Explained Variance Score:", evs_svr)

4/4 [==============================] - 0s 26ms/step
CNN Model Performance:
Mean Squared Error: 0.09319740064890852
Mean Absolute Error: 0.22319658758486316
R-squared: 0.998383100412454
Explained Variance Score: 0.9984813799278692
CNN + SVR Model Performance:
Mean Squared Error: 0.06118721200610771
Mean Absolute Error: 0.17601846266145332
R-squared: 0.9989384513176665
Explained Variance Score: 0.9989441678755333


In [79]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


import xgboost as xgb
# XGBoost model
xgb_predictions = xgb_model.predict(feat_test)

mse_xgb = mean_squared_error(Y_testset1, xgb_predictions)
mae_xgb = mean_absolute_error(Y_testset1, xgb_predictions)
r2_xgb = r2_score(Y_testset1, xgb_predictions)
evs_xgb = explained_variance_score(Y_testset1, xgb_predictions)

print("CNN + XGBoost Model Performance:")
print("Mean Squared Error:", mse_xgb)
print("Mean Absolute Error:", mae_xgb)
print("R-squared:", r2_xgb)
print("Explained Variance Score:", evs_xgb)




24/24 [==============================] - 1s 48ms/step
CNN Model Performance:
Mean Squared Error: 38.32944974198886
Mean Absolute Error: 3.377752460176816
R-squared: 0.03132606737308663
Explained Variance Score: 0.27612939398497494
CNN + XGBoost Model Performance:
Mean Squared Error: 0.015856978499013553
Mean Absolute Error: 0.018752507077279563
R-squared: 0.9997248942372184
Explained Variance Score: 0.9997250425659743


In [80]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.ensemble import RandomForestRegressor
# Random Forest model
rf_predictions = rf_model.predict(feat_test)

mse_rf = mean_squared_error(Y_testset1, rf_predictions)
mae_rf = mean_absolute_error(Y_testset1, rf_predictions)
r2_rf = r2_score(Y_testset1, rf_predictions)
evs_rf = explained_variance_score(Y_testset1, rf_predictions)

print("CNN + Random Forest Model Performance:")
print("Mean Squared Error:", mse_rf)
print("Mean Absolute Error:", mae_rf)
print("R-squared:", r2_rf)
print("Explained Variance Score:", evs_rf)




24/24 [==============================] - 1s 50ms/step
CNN Model Performance:
Mean Squared Error: 38.32944974198886
Mean Absolute Error: 3.377752460176816
R-squared: 0.03132606737308663
Explained Variance Score: 0.27612939398497494
CNN + Random Forest Model Performance:
Mean Squared Error: 0.004389587487574698
Mean Absolute Error: 0.01819194006097952
R-squared: 0.9999238442043583
Explained Variance Score: 0.99992481728793


In [81]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.tree import DecisionTreeRegressor

dt_predictions = dt_model.predict(feat_test)

mse_dt = mean_squared_error(Y_testset1, dt_predictions)
mae_dt = mean_absolute_error(Y_testset1, dt_predictions)
r2_dt = r2_score(Y_testset1, dt_predictions)
evs_dt = explained_variance_score(Y_testset1, dt_predictions)

print("CNN + Decision Tree Model Performance:")
print("Mean Squared Error:", mse_dt)
print("Mean Absolute Error:", mae_dt)
print("R-squared:", r2_dt)
print("Explained Variance Score:", evs_dt)



24/24 [==============================] - 1s 55ms/step
CNN Model Performance:
Mean Squared Error: 38.32944974198886
Mean Absolute Error: 3.377752460176816
R-squared: 0.03132606737308663
Explained Variance Score: 0.27612939398497494
CNN + Decision Tree Model Performance:
Mean Squared Error: 1.0110567268582634e-29
Mean Absolute Error: 2.1908401019269757e-15
R-squared: 1.0
Explained Variance Score: 1.0


In [82]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

from sklearn.linear_model import SGDRegressor

# SGD model
sgd_predictions = sgd_model.predict(feat_test)

mse_sgd = mean_squared_error(Y_testset1, sgd_predictions)
mae_sgd = mean_absolute_error(Y_testset1, sgd_predictions)
r2_sgd = r2_score(Y_testset1, sgd_predictions)
evs_sgd = explained_variance_score(Y_testset1, sgd_predictions)

print("CNN + SGD Model Performance:")
print("Mean Squared Error:", mse_sgd)
print("Mean Absolute Error:", mae_sgd)
print("R-squared:", r2_sgd)
print("Explained Variance Score:", evs_sgd)



24/24 [==============================] - 1s 54ms/step
CNN Model Performance:
Mean Squared Error: 38.32944974198886
Mean Absolute Error: 3.377752460176816
R-squared: 0.03132606737308663
Explained Variance Score: 0.27612939398497494
CNN + SGD Model Performance:
Mean Squared Error: 0.9250952838729312
Mean Absolute Error: 0.820755536618218
R-squared: 0.9839503444031694
Explained Variance Score: 0.9956176981666468


In [83]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
import os

# Specify the model folder
model_folder = 'conv_classifier_regressor'
results_folder = os.path.join(folder, 'results', model_folder)
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)
    print('created folder', results_folder)

# Define the CNN model architecture
input_0 = Input(shape=(X_trainset2.shape[1], X_trainset2.shape[2], 1))
conv_0 = Conv2D(4, kernel_size=(3, 3), activation='relu')(input_0)
pool_0 = MaxPooling2D(pool_size=(2, 2))(conv_0)

conv_1 = Conv2D(8, kernel_size=(3, 3), activation='relu')(pool_0)
pool_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)

conv_2 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)

conv_3 = Conv2D(32, kernel_size=(3, 3), activation='relu')(pool_2)
pool_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)

flat_0 = Flatten()(pool_3)
dense_0 = Dense(1024, activation='relu')(flat_0)
dense_1_0 = Dense(512, activation='relu', name='bisect')(dense_0)

out_0 = Dense(1, name='evm')(dense_1_0)
cnn_model = Model(inputs=input_0, outputs=out_0)
cnn_model.compile(optimizer=optimizers.Adam(lr=1e-04), loss='msle')
cnn_model.summary()

# Split the data into train and test sets
X_train_cnn, X_test_cnn, Y_train_cnn, Y_test_cnn = train_test_split(X_trainset2, Y_trainset2, test_size=0.2, random_state=42)

# Train the CNN model
cnn_model.fit(X_train_cnn, Y_train_cnn, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate CNN model
Y_pred_cnn = cnn_model.predict(X_testset2)

mse_cnn = mean_squared_error(Y_testset2, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_testset2, Y_pred_cnn)
r2_cnn = r2_score(Y_testset2, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_testset2, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 420, 420, 1)]     0         
                                                                 
 conv2d_16 (Conv2D)          (None, 418, 418, 4)       40        
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 209, 209, 4)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 207, 207, 8)       296       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 103, 103, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 101, 101, 16)      1168

In [84]:
model_feat = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer("bisect").output)
feat_train = model_feat.predict(X_trainset2)
print(feat_train.shape)

11/11 [==============================] - 1s 46ms/step
(325, 512)


In [85]:
feat_test = model_feat.predict(X_testset2)
print(feat_test.shape)

6/6 [==============================] - 0s 29ms/step
(162, 512)


In [86]:
from timeit import default_timer as timer
from sklearn.svm import SVR

# Create SVR model
svr_model = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svr_model.fit(feat_train, Y_trainset2)

# Start time
start = timer()
svr_predictions_train = svr_model.predict(feat_train)
# End time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.00978255644440651


In [87]:
import xgboost as xgb

# Create XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(feat_train, Y_trainset2)

# Start time
start = timer()
xgb_predictions_train = xgb_model.predict(feat_train)
# End time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.0017330124974250793


In [88]:
from sklearn.ensemble import RandomForestRegressor

# Create Random Forest model
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(feat_train, Y_trainset2)

# Start time
start = timer()
rf_predictions_train = rf_model.predict(feat_train)
# End time
end = timer()
print("Time taken to predict with trained data: ", end - start)



Time taken to predict with trained data:  0.006910894066095352


In [89]:
from sklearn.tree import DecisionTreeRegressor

# Create Decision Tree model
dt_model = DecisionTreeRegressor()
dt_model.fit(feat_train, Y_trainset2)

# Start time
start = timer()
dt_predictions_train = dt_model.predict(feat_train)
# End time
end = timer()
print("Time taken to predict with trained data: ", end - start)


Time taken to predict with trained data:  0.0002777799963951111


In [90]:
from sklearn.linear_model import SGDRegressor

# Create SGD model
sgd_model = SGDRegressor(loss='epsilon_insensitive', epsilon=0.1, max_iter=1000, tol=1e-3)
sgd_model.fit(feat_train, Y_trainset2)

# Start time
start = timer()
sgd_predictions_train = sgd_model.predict(feat_train)
# End time
end = timer()
print("Time taken to predict with trained data: ", end - start)

Time taken to predict with trained data:  0.0006718654185533524


In [91]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR

# Evaluate CNN model
Y_pred_cnn = cnn_model.predict(X_testset2)

mse_cnn = mean_squared_error(Y_testset2, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_testset2, Y_pred_cnn)
r2_cnn = r2_score(Y_testset2, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_testset2, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

# SVR model
svr_predictions = svr_model.predict(feat_test)

mse_combined = mean_squared_error(Y_testset2, svr_predictions)
mae_combined = mean_absolute_error(Y_testset2, svr_predictions)
r2_combined = r2_score(Y_testset2, svr_predictions)
evs_combined = explained_variance_score(Y_testset2, svr_predictions)

print("CNN + SVR Model Performance:")
print("Mean Squared Error:", mse_combined)
print("Mean Absolute Error:", mae_combined)
print("R-squared:", r2_combined)
print("Explained Variance Score:", evs_combined)


6/6 [==============================] - 0s 40ms/step
CNN Model Performance:
Mean Squared Error: 30.926860221233635
Mean Absolute Error: 5.5236068448718125
R-squared: -73.80567866461641
Explained Variance Score: -0.007736106275642252
CNN + SVR Model Performance:
Mean Squared Error: 0.1745318762650198
Mean Absolute Error: 0.2557150484820016
R-squared: 0.5778434875309539
Explained Variance Score: 0.6657948164384937


In [92]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

import xgboost as xgb

# XGBoost model
xgb_predictions = xgb_model.predict(feat_test)

mse_xgb = mean_squared_error(Y_testset2, xgb_predictions)
mae_xgb = mean_absolute_error(Y_testset2, xgb_predictions)
r2_xgb = r2_score(Y_testset2, xgb_predictions)
evs_xgb = explained_variance_score(Y_testset2, xgb_predictions)

print("CNN + XGBoost Model Performance:")
print("Mean Squared Error:", mse_xgb)
print("Mean Absolute Error:", mae_xgb)
print("R-squared:", r2_xgb)
print("Explained Variance Score:", evs_xgb)


24/24 [==============================] - 1s 56ms/step
CNN Model Performance:
Mean Squared Error: 112.10705511133666
Mean Absolute Error: 8.518157583500468
R-squared: -1.8332048250345099
Explained Variance Score: 0.0005293062405008486
CNN + XGBoost Model Performance:
Mean Squared Error: 0.20956755622341902
Mean Absolute Error: 0.2690156579756964
R-squared: 0.493099423697247
Explained Variance Score: 0.6321136622720358


In [93]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.ensemble import RandomForestRegressor

# Random Forest model
rf_predictions = rf_model.predict(feat_test)

mse_rf = mean_squared_error(Y_testset2, rf_predictions)
mae_rf = mean_absolute_error(Y_testset2, rf_predictions)
r2_rf = r2_score(Y_testset2, rf_predictions)
evs_rf = explained_variance_score(Y_testset2, rf_predictions)

print("CNN + Random Forest Model Performance:")
print("Mean Squared Error:", mse_rf)
print("Mean Absolute Error:", mae_rf)
print("R-squared:", r2_rf)
print("Explained Variance Score:", evs_rf)


24/24 [==============================] - 1s 56ms/step
CNN Model Performance:
Mean Squared Error: 112.10705511133666
Mean Absolute Error: 8.518157583500468
R-squared: -1.8332048250345099
Explained Variance Score: 0.0005293062405008486
CNN + Random Forest Model Performance:
Mean Squared Error: 0.23979326374839308
Mean Absolute Error: 0.27930844529439014
R-squared: 0.41998968839435646
Explained Variance Score: 0.5803252669809746


In [94]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)


from sklearn.tree import DecisionTreeRegressor

# Decision Tree model
dt_predictions = dt_model.predict(feat_test)

mse_dt = mean_squared_error(Y_testset2, dt_predictions)
mae_dt = mean_absolute_error(Y_testset2, dt_predictions)
r2_dt = r2_score(Y_testset2, dt_predictions)
evs_dt = explained_variance_score(Y_testset2, dt_predictions)

print("CNN + Decision Tree Model Performance:")
print("Mean Squared Error:", mse_dt)
print("Mean Absolute Error:", mae_dt)
print("R-squared:", r2_dt)
print("Explained Variance Score:", evs_dt)


24/24 [==============================] - 1s 52ms/step
CNN Model Performance:
Mean Squared Error: 112.10705511133666
Mean Absolute Error: 8.518157583500468
R-squared: -1.8332048250345099
Explained Variance Score: 0.0005293062405008486
CNN + Decision Tree Model Performance:
Mean Squared Error: 0.24930452315616017
Mean Absolute Error: 0.2868246553367966
R-squared: 0.39698391898021024
Explained Variance Score: 0.5415055373665545


In [95]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVR


# Evaluate CNN model 
Y_pred_cnn = cnn_model.predict(X_test)

mse_cnn = mean_squared_error(Y_test, Y_pred_cnn)
mae_cnn = mean_absolute_error(Y_test, Y_pred_cnn)
r2_cnn = r2_score(Y_test, Y_pred_cnn)
evs_cnn = explained_variance_score(Y_test, Y_pred_cnn)

print("CNN Model Performance:")
print("Mean Squared Error:", mse_cnn)
print("Mean Absolute Error:", mae_cnn)
print("R-squared:", r2_cnn)
print("Explained Variance Score:", evs_cnn)

from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import SGDRegressor

# SGD model
sgd_predictions = sgd_model.predict(feat_test)

mse_sgd = mean_squared_error(Y_testset2, sgd_predictions)
mae_sgd = mean_absolute_error(Y_testset2, sgd_predictions)
r2_sgd = r2_score(Y_testset2, sgd_predictions)
evs_sgd = explained_variance_score(Y_testset2, sgd_predictions)

print("CNN + SGD Model Performance:")
print("Mean Squared Error:", mse_sgd)
print("Mean Absolute Error:", mae_sgd)
print("R-squared:", r2_sgd)
print("Explained Variance Score:", evs_sgd)



24/24 [==============================] - 1s 61ms/step
CNN Model Performance:
Mean Squared Error: 112.10705511133666
Mean Absolute Error: 8.518157583500468
R-squared: -1.8332048250345099
Explained Variance Score: 0.0005293062405008486
CNN + SGD Model Performance:
Mean Squared Error: 0.515988452452287
Mean Absolute Error: 0.5150666576976075
R-squared: -0.24806935112984507
Explained Variance Score: 0.09781133927058672
